In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import optuna
import math
import time
import seaborn as sns
#from optuna.integration.dask import DaskStorage
import networkx as nx
from tqdm import tqdm
from scipy import io
from tqdm import tqdm
from matplotlib import pyplot as plt
from dask.distributed import Client, progress, wait
from IPython.display import display, HTML
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix
from sklearn.mixture import GaussianMixture
from scipy.optimize import minimize_scalar
from scipy import stats
from sklearn.cluster import KMeans
from scipy.stats import norm
from sklearn.metrics import silhouette_score

import project_path
from src.util.t2m import t2m
from src.util.m2t import m2t
# from src.algos.lr_stss import lr_stss
from src.algos.lr_stss_modified import lr_stss_modified
from src.algos.lr_sts import lr_sts_hard
from src.algos.lr_sss import lr_sss_hard
from src.algos.horpca_singleton import horpca_singleton
from dask.distributed import as_completed

In [2]:
# Load pre-processed Data
cwd = os.getcwd()
data_dir = os.path.join(cwd,'data','nyc')
zones = gpd.read_file(os.path.join(data_dir,'taxi_zones_shapefile','taxi_zones.shp'))
zone_lookup = os.path.join(data_dir, 'taxi_zone_lookup.csv')
zone_lookup = pd.read_csv(zone_lookup)

pickups = np.load(os.path.join(data_dir,"hourly_pickup.npy"))
dropoffs = np.load(os.path.join(data_dir,"hourly_dropoff.npy"))
# # Load Emre's settings
dates = io.loadmat(os.path.join(data_dir,'dates.mat'))
regions = io.loadmat(os.path.join(data_dir,'regions.mat'))
neighbors = io.loadmat(os.path.join(data_dir,'neighbors.mat'))
regions=regions['regions'].ravel()
arrivals = io.loadmat('arrivals.mat')['Y']
# # Filter the data
# pickups = pickups[regions-1, ...]
# dropoffs = dropoffs[regions-1, ...]
zones = zones.iloc[regions-1]
# dos = np.zeros((81,53,7,24))
# dos[:,:52,:,:] = dropoffs
# dos[:,52,1:,:] = np.mean(dropoffs[:,:,1:,:],1)
# dropoffs = dos.copy()
# dos[:,:52,:,:] = pickups
# dos[:,52,1:,:] = np.mean(pickups[:,:,1:,:],1)
# pickups = dos.copy()
# del dos

dropoffs = np.moveaxis(arrivals,[0,1,2,3],[3,2,1,0])

In [3]:
pos = np.zeros((81,2))
pos[:,0] = zones.geometry.centroid.x.values
pos[:,1] = zones.geometry.centroid.y.values

position ={}
G_nyc = nx.Graph()
G_nyc.add_nodes_from([(regions[i], {'pos': pos[i,:], 'LocationID': regions[i], 'zone': zones.iloc[i]['zone']}) for i in range(81)])
edge_list =[]
for i in range(len(neighbors['neighbors'].ravel())):
    for neighbor in neighbors['neighbors'].ravel()[i].ravel():
        if np.any(np.isin(neighbor, regions)) and (neighbor!=regions[i]):
            edge_list.append((regions[i], neighbor))
    # edge_list = edge_list + [(regions[i], neighbor) for neighbor in neighbors['neighbors'].ravel()[i]]
    position[regions[i]]=pos[i,:]
G_nyc.add_edges_from(edge_list)
# G_nyc.nodes()
A = nx.adjacency_matrix(G_nyc).toarray()
Deg = np.diag(np.asarray(np.sum(A,axis=1)).ravel())
Dsq = np.linalg.inv(np.sqrt(Deg))
An = Dsq@A@Dsq

In [4]:
def detect_topk_events(anomaly_scores, ratio):
#     events_start_ts = pd.to_datetime(['01-Jan-2018', '03-Jan-2018 16:00:00', '14-Jan-2018 09:00:00', '20-Jan-2018 08:00:00', 
#                                     '28-Jan-2018 16:00:00', '04-Mar-2018 15:00:00', '31-Mar-2018 13:00:00', '17-Mar-2018 11:00:00',
#                                     '20-Mar-2018 10:00:00', '21-Mar-2018 16:00:00', '01-Jul-2018 17:00:00', '04-Jul-2018 17:00:00',
#                                     '25-Sep-2018 10:00:00', '04-Oct-2018 08:00:00', '04-Nov-2018 12:00:00', '09-Nov-2018 19:00:00',
#                                     '22-Nov-2018 21:00:00', '04-Dec-2018 19:00:00', '16-Dec-2018 10:00:00', '31-Dec-2018 21:00:00'
#                                     ])
    events_start_ts = pd.to_datetime(['01-Jan-2018 00:00:00', '03-Jan-2018 16:00:00', '14-Jan-2018 09:00:00', '20-Jan-2018 08:00:00', 
                                    '4-Mar-2018 15:00:00', '08-Mar-2018 18:00:00', '17-Mar-2018 11:00:00', '20-Mar-2018 10:00:00',
                                    '21-Mar-2018 16:00:00', '01-Jul-2018 17:00:00', '04-Jul-2018 17:00:00', '25-Sep-2018 10:00:00',
                                    '04-Oct-2018 08:00:00', '04-Nov-2018 12:00:00', '09-Nov-2018 19:00:00', '22-Nov-2018 21:00:00',
                                    '4-Dec-2018 19:00:00', '16-Dec-2018 10:00:00', '28-Dec-2018 12:00:00', '31-Dec-2018 21:00:00'
    ])
#     events_end_ts = pd.to_datetime(['01-Jan-2018 02:00:00', '03-Jan-2018 22:00:00', '14-Jan-2018 17:00:00', '20-Jan-2018 15:00:00',
#                                 '28-Jan-2018 23:00:00', '04-Mar-2018 22:00:00', '31-Mar-2018 20:00:00', '17-Mar-2018 17:00:00',
#                                 '20-Mar-2018 20:00:00', '21-Mar-2018 22:00:00', '01-Jul-2018 22:00:00', '04-Jul-2018 23:00:00',
#                                 '25-Sep-2018 20:00:00', '04-Oct-2018 15:00:00', '04-Nov-2018 17:00:00', '09-Nov-2018 23:30:00',
#                                 '22-Nov-2018 23:59:00', '04-Dec-2018 23:59:00', '16-Dec-2018 15:00:00', '31-Dec-2018 23:59:00'
#     ])
    events_end_ts = pd.to_datetime(['01-Jan-2018 02:00:00', '03-Jan-2018 22:00:00', '14-Jan-2018 17:00:00', '20-Jan-2018 15:00:00',
                             '4-Mar-2018 22:00:00', '08-Mar-2018 23:59:00', '17-Mar-2018 17:00:00', '20-Mar-2018 20:00:00',
                            '21-Mar-2018 22:00:00', '01-Jul-2018 22:00:00', '04-Jul-2018 23:00:00', '25-Sep-2018 20:00:00',
                            '04-Oct-2018 15:00:00', '04-Nov-2018 17:00:00', '09-Nov-2018 23:30:00', '22-Nov-2018 23:59:00',
                            '4-Dec-2018 23:59:00', '16-Dec-2018 15:00:00', '28-Dec-2018 17:00:00', '31-Dec-2018 23:59:00'
    ])
    indd = np.flip(np.argsort(anomaly_scores, axis=None))
    ind = np.unravel_index(indd[:int(len(indd)*ratio)], anomaly_scores.shape)
    topk_event_idx = ind
    anomaly_mask = np.zeros(anomaly_scores.shape, dtype=bool)
    anomaly_mask[topk_event_idx] =1
#     num_detected_events = 0
    detected_events = np.zeros(20)

    idxs = np.arange(81)
    # w = events_start_ts.isocalendar().week
    # d = events_start_ts.day_of_week
    doy = events_start_ts.day_of_year
    w = (doy-1)//(7)
    d = (doy-1) % 7
    h_s = events_start_ts.hour
    h_e = events_end_ts.hour
    for i in range(20):
        event_mask = np.zeros(anomaly_scores.shape, dtype=bool)
        locations = dates['dates'][2].ravel()[i].ravel()
        
        for loc in locations: 
            # event_mask[idxs[regions==loc], w[i]-1, d[i], h_s[i]:h_e[i]] = 1
            # event_mask[idxs[regions==loc], w[i]-1, d[i], h_e[i]] = 1
            event_mask[idxs[regions==loc], w[i], d[i], h_s[i]:h_e[i]] = 1
            event_mask[idxs[regions==loc], w[i], d[i], h_e[i]] = 1
        if np.any(event_mask * anomaly_mask):
#             num_detected_events +=1
            detected_events[i]=1
    return detected_events#num_detected_events

In [6]:
def detect_events(anomaly_mask):
    events_start_ts = pd.to_datetime(['01-Jan-2018 00:00:00', '03-Jan-2018 16:00:00', '14-Jan-2018 09:00:00', '20-Jan-2018 08:00:00', 
                                    '4-Mar-2018 15:00:00', '08-Mar-2018 18:00:00', '17-Mar-2018 11:00:00', '20-Mar-2018 10:00:00',
                                    '21-Mar-2018 16:00:00', '01-Jul-2018 17:00:00', '04-Jul-2018 17:00:00', '25-Sep-2018 10:00:00',
                                    '04-Oct-2018 08:00:00', '04-Nov-2018 12:00:00', '09-Nov-2018 19:00:00', '22-Nov-2018 21:00:00',
                                    '4-Dec-2018 19:00:00', '16-Dec-2018 10:00:00', '28-Dec-2018 12:00:00', '31-Dec-2018 21:00:00'
    ])
    events_end_ts = pd.to_datetime(['01-Jan-2018 02:00:00', '03-Jan-2018 22:00:00', '14-Jan-2018 17:00:00', '20-Jan-2018 15:00:00',
                             '4-Mar-2018 22:00:00', '08-Mar-2018 23:59:00', '17-Mar-2018 17:00:00', '20-Mar-2018 20:00:00',
                            '21-Mar-2018 22:00:00', '01-Jul-2018 22:00:00', '04-Jul-2018 23:00:00', '25-Sep-2018 20:00:00',
                            '04-Oct-2018 15:00:00', '04-Nov-2018 17:00:00', '09-Nov-2018 23:30:00', '22-Nov-2018 23:59:00',
                            '4-Dec-2018 23:59:00', '16-Dec-2018 15:00:00', '28-Dec-2018 17:00:00', '31-Dec-2018 23:59:00'
    ])
    # indd = np.flip(np.argsort(anomaly_scores, axis=None))
    # ind = np.unravel_index(indd[:int(len(indd)*ratio)], anomaly_scores.shape)
    # topk_event_idx = ind
    # anomaly_mask = np.zeros(anomaly_scores.shape, dtype=bool)
    # anomaly_mask[topk_event_idx] =1
#     num_detected_events = 0
    detected_events = np.zeros(20)

    idxs = np.arange(81)
    # w = events_start_ts.isocalendar().week
    # d = events_start_ts.day_of_week
    doy = events_start_ts.day_of_year
    w = (doy-1)//(7)
    d = (doy-1) % 7
    h_s = events_start_ts.hour
    h_e = events_end_ts.hour
    for i in range(20):
        event_mask = np.zeros(anomaly_mask.shape, dtype=bool)
        locations = dates['dates'][2].ravel()[i].ravel()
        
        for loc in locations: 
            # event_mask[idxs[regions==loc], w[i]-1, d[i], h_s[i]:h_e[i]] = 1
            # event_mask[idxs[regions==loc], w[i]-1, d[i], h_e[i]] = 1
            event_mask[idxs[regions==loc], w[i], d[i], h_s[i]:h_e[i]] = 1
            event_mask[idxs[regions==loc], w[i], d[i], h_e[i]] = 1
        if np.any(event_mask * anomaly_mask):
#             num_detected_events +=1
            detected_events[i]=1
    return detected_events#num_detected_events

## LRSTSS

In [13]:
psi =  6.330030868935934 # 12.83335528216673 # 32# final: psi = 6.330030868935934
lda_l = 52.4203154644308 # 6.386243343144918 # 56#      lda_l = 
lda_1 = 381.5726253552148 # 471.62025607182056 # 48#.
lda_t = 0.05662389826485177 # 251.84760832599633# 100 #
rho = 2.8422507662618838e-5 # 5.4622401816620816e-5 # 0.0005 #
lda_2 = 1000
time_m = 4
local_m = 1
maxit = 300
psi_dist = np.array([0.83768957, 1.        , 0.36744661, 0.64250066])
res_stss = lr_stss_modified(dropoffs, An, time_m,local_m, verbose=1, max_it=maxit,
        lda2=lda_2, lda1=lda_1, lda_t=lda_t,
        lda_loc=lda_l, psis=psi_dist*psi, rho=rho, rho_upd=-1)
abs_s = np.abs(res_stss['S'])
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
num_detected_events = np.array([sum(detect_topk_events(abs_s, r)) for r in ratios])
detected_events = detect_topk_events(abs_s,0.03)
print(f"Detected 3% events:\t {detected_events}")
print(f"Number of detected events:\t {num_detected_events}")
# io.savemat('nyc_taxi_lrstss_1.mat', {'S':np.moveaxis(res_stss['S'],[0,1,2,3],[3,2,1,0])})

It-2:	## |r|=103515.93550 	 ## |s|=512933.41201 	 ## rho=0.0000 obj=2772001.2452 	 ## del_obj = 2441824.9707 
It-3:	## |r|=86024.95507 	 ## |s|=82028.24821 	 ## rho=0.0000 obj=3194511.7546 	 ## del_obj = 422510.5094 
It-4:	## |r|=76924.19091 	 ## |s|=34644.39591 	 ## rho=0.0000 obj=3317128.8125 	 ## del_obj = 122617.0579 
It-5:	## |r|=67919.20059 	 ## |s|=33463.35907 	 ## rho=0.0000 obj=3475098.2214 	 ## del_obj = 157969.4089 
It-6:	## |r|=61551.51709 	 ## |s|=32141.34092 	 ## rho=0.0000 obj=3632757.6537 	 ## del_obj = 157659.4324 
It-7:	## |r|=54408.37709 	 ## |s|=28344.60347 	 ## rho=0.0000 obj=3760573.4224 	 ## del_obj = 127815.7687 
It-8:	## |r|=51302.67958 	 ## |s|=20984.79314 	 ## rho=0.0000 obj=3857522.9356 	 ## del_obj = 96949.5132 
It-9:	## |r|=44904.83616 	 ## |s|=27150.61203 	 ## rho=0.0000 obj=3992771.9055 	 ## del_obj = 135248.9699 
It-10:	## |r|=44153.87751 	 ## |s|=8378.65866 	 ## rho=0.0000 obj=4031297.7796 	 ## del_obj = 38525.8741 
It-11:	## |r|=43232.92332 	 ## |s|=7

In [15]:
S = res_stss['S']
S_loc = t2m(S, m= 1)
r_A = np.eye(S_loc.shape[0])+np.linalg.matrix_power(A,3)
likelihood = np.zeros(S_loc.shape)
for s in range(S_loc.shape[0]):
    mask = r_A[np.where(r_A[s,]!=0),:].astype(bool)
    nbd = S_loc[mask[:,0,:].ravel(),:]
    # Append neighbors from additional columns in mask
    for m in range(1, mask.shape[1]):
        nbd = np.vstack((nbd,S_loc[mask[:,m,:].ravel(),:] ))
        
    W = np.zeros(nbd.shape)
    # Iterate through the columns in steps of block_size
    for i1 in range(0, 8904, 168):
        # Slice the matrix to get the block (columns from i to i + block_size)
        if i1==0:
            block = nbd[:, i1:i1 + 168]
            for loc in range(nbd.shape[0]):
                W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1:i1+168]),0,30)
        elif i1==8736:
            block = nbd[:, i1-168:i1 + 168]
            for loc in range(nbd.shape[0]):
                W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+168]),0,30)
        else:
            block = nbd[:, i1-168:i1 + 336]
            for loc in range(nbd.shape[0]):
                W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+336]),0,30)
    mean = np.sum(W * nbd) / np.sum(W)
    sd = np.sqrt(np.sum(W * (nbd - mean)**2) / np.sum(W))
    likelihood[s,] = np.log(sd) + (0.5*np.power(((S_loc[s,] - mean)/sd),2))


In [17]:
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3, 4, 5])/100
LRTSS_detected_events = []
for k in ratios:
    Q = np.quantile(likelihood, (1-k))
    anom = np.zeros(S_loc.shape)
    anom[likelihood> Q] = 1
    #anom[LR<Q1]=1
    anom = anom.reshape(-1,1)
    detected_events = detect_events(anom.reshape(abs_s.shape))
    LRTSS_detected_events.append({'top-k%': k*100, 'Detected_Events': sum(detected_events)})



In [19]:
LRTSS_detected_events

[{'top-k%': 0.014000000000000002, 'Detected_Events': 3.0},
 {'top-k%': 0.07, 'Detected_Events': 5.0},
 {'top-k%': 0.14, 'Detected_Events': 9.0},
 {'top-k%': 0.3, 'Detected_Events': 12.0},
 {'top-k%': 0.7, 'Detected_Events': 15.0},
 {'top-k%': 1.0, 'Detected_Events': 16.0},
 {'top-k%': 2.0, 'Detected_Events': 19.0},
 {'top-k%': 3.0, 'Detected_Events': 19.0},
 {'top-k%': 4.0, 'Detected_Events': 20.0},
 {'top-k%': 5.0, 'Detected_Events': 20.0}]

In [23]:
num_detected_events = np.array([sum(detect_topk_events(abs_s, r)) for r in ratios])
num_detected_events

array([ 3.,  5.,  7., 11., 15., 16., 18., 19., 19., 19.])

In [26]:
df = pd.DataFrame(LRTSS_detected_events)
df.to_csv('NYC_LRTSS_detected_events.csv', index=False)

## LRTS

In [25]:
time_m = 4
local_m = 1
maxit = 300
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
lda_1 = 6.383620843871507
lda_t = 0.0015494422845345637
rho = 0.00014877031119314767
# lda_1 = 2.21133835238238; lda_t = 0.0003254198184799227; rho= 0.00013934824565267574;
res_sts = lr_sts_hard(dropoffs, time_m, verbose=1, max_it=maxit,
        lda1=lda_1, lda_t=lda_t,
        psis=psi_dist, rho=rho)
abs_s = np.abs(res_sts['S'])
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
num_detected_events = np.array([sum(detect_topk_events(abs_s, r)) for r in ratios])
detected_events = detect_topk_events(abs_s,0.03)
print(f"Detected 3% events:\t {detected_events}")
print(f"Number of detected events:\t {num_detected_events}")
# io.savemat('nyc_taxi_lrsts_2.mat', {'S':np.moveaxis(res_sts['S'],[0,1,2,3],[3,2,1,0])})

It-2:	## |r|=88809.42054 	 ## |s|=51974.38877 	 ## rho=0.0001 obj=203609948.5626 	 ## del_obj = -33649468.6073 
It-3:	## |r|=91103.90025 	 ## |s|=60087.49418 	 ## rho=0.0001 obj=54177482.3704 	 ## del_obj = -149432466.1922 
It-4:	## |r|=70742.52460 	 ## |s|=60583.40123 	 ## rho=0.0001 obj=26201063.1688 	 ## del_obj = -27976419.2015 
It-5:	## |r|=74671.13614 	 ## |s|=49991.68876 	 ## rho=0.0001 obj=63035432.0210 	 ## del_obj = 36834368.8522 
It-6:	## |r|=56901.06046 	 ## |s|=45358.95881 	 ## rho=0.0001 obj=67272730.6827 	 ## del_obj = 4237298.6617 
It-7:	## |r|=55467.36931 	 ## |s|=29713.18346 	 ## rho=0.0001 obj=93595621.6655 	 ## del_obj = 26322890.9828 
It-8:	## |r|=42370.27494 	 ## |s|=23930.48881 	 ## rho=0.0001 obj=77203965.9464 	 ## del_obj = -16391655.7191 
It-9:	## |r|=38996.55732 	 ## |s|=10764.69776 	 ## rho=0.0001 obj=75094438.6917 	 ## del_obj = -2109527.2548 
It-10:	## |r|=31630.05402 	 ## |s|=8937.29324 	 ## rho=0.0001 obj=52015104.5366 	 ## del_obj = -23079334.1550 
It-1

In [27]:
        S = res_sts['S']

        S_loc = t2m(S, m= 1)
        r_A = np.eye(S_loc.shape[0])+np.linalg.matrix_power(A,3)
        likelihood = np.zeros(S_loc.shape)
        for s in range(S_loc.shape[0]):
            mask = r_A[np.where(r_A[s,]!=0),:].astype(bool)
            nbd = S_loc[mask[:,0,:].ravel(),:]
            # Append neighbors from additional columns in mask
            for m in range(1, mask.shape[1]):
                nbd = np.vstack((nbd,S_loc[mask[:,m,:].ravel(),:] ))
                
            W = np.zeros(nbd.shape)
            # Iterate through the columns in steps of block_size
            for i1 in range(0, 8904, 168):
                # Slice the matrix to get the block (columns from i to i + block_size)
                if i1==0:
                    block = nbd[:, i1:i1 + 168]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1:i1+168]),0,30)
                elif i1==8736:
                    block = nbd[:, i1-168:i1 + 168]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+168]),0,30)
                else:
                    block = nbd[:, i1-168:i1 + 336]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+336]),0,30)
            mean = np.sum(W * nbd) / np.sum(W)
            sd = np.sqrt(np.sum(W * (nbd - mean)**2) / np.sum(W))
            likelihood[s,] = np.log(sd) + (0.5*np.power(((S_loc[s,] - mean)/sd),2))
        

In [28]:
    ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3, 4, 5])/100
    LRTS_detected_events = []
    for k1 in ratios:
        Q = np.quantile(likelihood, (1-k1))
        anom = np.zeros(S_loc.shape)
        anom[likelihood> Q] = 1
        #anom[LR<Q1]=1
        anom = anom.reshape(-1,1)
        detected_events = detect_events(anom.reshape(abs_s.shape))
        LRTS_detected_events.append({'top-k%': k1*100, 'Detected_Events': sum(detected_events)})
    
    

In [29]:
df = pd.DataFrame(LRTS_detected_events)
df.to_csv('NYC_LRTS_detected_events.csv', index=False)

In [30]:
df

,top-k%,Detected_Events
0,0.014,2.0
1,0.070,4.0
2,0.140,4.0
3,0.300,6.0
4,0.700,8.0
5,1.000,11.0
6,2.000,13.0
7,3.000,17.0
8,4.000,19.0
9,5.000,20.0


In [31]:
num_detected_events = np.array([sum(detect_topk_events(abs_s, r)) for r in ratios])
num_detected_events

array([ 2.,  4.,  4.,  6.,  8., 11., 13., 17., 19., 20.])

## LRSS

In [33]:
time_m = 4
local_m = 1
maxit = 300
psi_dist = np.array([0.83150206, 1, 0.36393466, 0.63659289])
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
lda_1 = 4.843400142487287; lda_l = 32.69839352044304; rho= 0.05492962719205545
res = lr_sss_hard(dropoffs, np.eye(dropoffs.shape[local_m-1]) - An, local_m, verbose=1, max_it=maxit,
        lda1=lda_1, rho=rho,
        lda_loc=lda_l, psis=psi_dist)
abs_s = np.abs(res['S'])
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
num_detected_events = np.array([detect_topk_events(abs_s, r) for r in ratios])
print(num_detected_events)

It-2:	## |r|=38953.70654 	 ## |s|=69213.06784 	 ## rho=0.0549 obj=1419466556.0050 	 ## del_obj = 332087124.6253 
It-3:	## |r|=62917.24966 	 ## |s|=28365.87007 	 ## rho=0.0549 obj=794528343.2226 	 ## del_obj = -624938212.7824 
It-4:	## |r|=34287.60958 	 ## |s|=29261.16655 	 ## rho=0.0549 obj=649699663.6180 	 ## del_obj = -144828679.6046 
It-5:	## |r|=30837.81946 	 ## |s|=26716.87060 	 ## rho=0.0549 obj=660124928.5920 	 ## del_obj = 10425264.9741 
It-6:	## |r|=27996.76070 	 ## |s|=24731.05550 	 ## rho=0.0549 obj=634721562.5856 	 ## del_obj = -25403366.0064 
It-7:	## |r|=25988.51803 	 ## |s|=22268.66669 	 ## rho=0.0549 obj=590975617.7017 	 ## del_obj = -43745944.8839 
It-8:	## |r|=23995.43860 	 ## |s|=20544.90246 	 ## rho=0.0549 obj=515643556.7408 	 ## del_obj = -75332060.9609 
It-9:	## |r|=22598.58645 	 ## |s|=19153.42562 	 ## rho=0.0549 obj=440338439.4436 	 ## del_obj = -75305117.2973 
It-10:	## |r|=21370.58942 	 ## |s|=18243.64512 	 ## rho=0.0549 obj=367122075.9244 	 ## del_obj = -7321

In [34]:
        S = res['S']

        S_loc = t2m(S, m= 1)
        r_A = np.eye(S_loc.shape[0])+np.linalg.matrix_power(A,3)
        likelihood = np.zeros(S_loc.shape)
        for s in range(S_loc.shape[0]):
            mask = r_A[np.where(r_A[s,]!=0),:].astype(bool)
            nbd = S_loc[mask[:,0,:].ravel(),:]
            # Append neighbors from additional columns in mask
            for m in range(1, mask.shape[1]):
                nbd = np.vstack((nbd,S_loc[mask[:,m,:].ravel(),:] ))
                
            W = np.zeros(nbd.shape)
            # Iterate through the columns in steps of block_size
            for i1 in range(0, 8904, 168):
                # Slice the matrix to get the block (columns from i to i + block_size)
                if i1==0:
                    block = nbd[:, i1:i1 + 168]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1:i1+168]),0,30)
                elif i1==8736:
                    block = nbd[:, i1-168:i1 + 168]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+168]),0,30)
                else:
                    block = nbd[:, i1-168:i1 + 336]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+336]),0,30)
            mean = np.sum(W * nbd) / np.sum(W)
            sd = np.sqrt(np.sum(W * (nbd - mean)**2) / np.sum(W))
            likelihood[s,] = np.log(sd) + (0.5*np.power(((S_loc[s,] - mean)/sd),2))
        

In [35]:
    ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3, 4, 5])/100
    LRSS_detected_events = []
    for k2 in ratios:
        Q = np.quantile(likelihood, (1-k2))
        anom = np.zeros(S_loc.shape)
        anom[likelihood> Q] = 1
        #anom[LR<Q1]=1
        anom = anom.reshape(-1,1)
        detected_events = detect_events(anom.reshape(abs_s.shape))
        LRSS_detected_events.append({'top-k%': k2*100, 'Detected_Events': sum(detected_events)})
    
    

In [36]:
df = pd.DataFrame(LRSS_detected_events)
df.to_csv('NYC_LRSS_detected_events.csv', index=False)

In [37]:
df

,top-k%,Detected_Events
0,0.014,1.0
1,0.070,1.0
2,0.140,1.0
3,0.300,2.0
4,0.700,3.0
5,1.000,4.0
6,2.000,7.0
7,3.000,13.0
8,4.000,15.0
9,5.000,16.0


In [38]:
num_detected_events = np.array([sum(detect_topk_events(abs_s, r)) for r in ratios])
num_detected_events

array([ 1.,  1.,  1.,  2.,  3.,  3.,  7., 13., 14., 16.])

## HORPCA

In [41]:
time_m = 4
local_m = 1
maxit = 300
res = horpca_singleton(dropoffs, maxit=maxit, verbose=0)
abs_s = np.abs(res['E'])
ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3])/100
num_detected_events = np.array([detect_topk_events(abs_s, r) for r in ratios])
print(num_detected_events)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1.]]


In [42]:
        S = res['E']

        S_loc = t2m(S, m= 1)
        r_A = np.eye(S_loc.shape[0])+np.linalg.matrix_power(A,2)
        likelihood = np.zeros(S_loc.shape)
        for s in range(S_loc.shape[0]):
            mask = r_A[np.where(r_A[s,]!=0),:].astype(bool)
            nbd = S_loc[mask[:,0,:].ravel(),:]
            # Append neighbors from additional columns in mask
            for m in range(1, mask.shape[1]):
                nbd = np.vstack((nbd,S_loc[mask[:,m,:].ravel(),:] ))
                
            W = np.zeros(nbd.shape)
            # Iterate through the columns in steps of block_size
            for i1 in range(0, 8904, 168):
                # Slice the matrix to get the block (columns from i to i + block_size)
                if i1==0:
                    block = nbd[:, i1:i1 + 168]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1:i1+168]),0,30)
                elif i1==8736:
                    block = nbd[:, i1-168:i1 + 168]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+168]),0,30)
                else:
                    block = nbd[:, i1-168:i1 + 336]
                    for loc in range(nbd.shape[0]):
                        W[loc, i1:i1+168] = norm.pdf(np.linalg.norm(block[loc,:] - S_loc[s,i1-168:i1+336]),0,30)
            mean = np.sum(W * nbd) / np.sum(W)
            sd = np.sqrt(np.sum(W * (nbd - mean)**2) / np.sum(W))
            likelihood[s,] = np.log(sd) + (0.5*np.power(((S_loc[s,] - mean)/sd),2))
        

In [43]:
    ratios = np.array([0.014, 0.07, 0.14, 0.3, 0.7, 1, 2, 3, 4, 5])/100
    HoRPCA_detected_events = []
    for k3 in ratios:
        Q = np.quantile(likelihood, (1-k3))
        anom = np.zeros(S_loc.shape)
        anom[likelihood> Q] = 1
        #anom[LR<Q1]=1
        anom = anom.reshape(-1,1)
        detected_events = detect_events(anom.reshape(abs_s.shape))
        HoRPCA_detected_events.append({'top-k%': k3*100, 'Detected_Events': sum(detected_events)})
    
    

In [44]:
df = pd.DataFrame(HoRPCA_detected_events)
df.to_csv('NYC_HoRCA_detected_events.csv', index=False)

In [45]:
df

,top-k%,Detected_Events
0,0.014,0.0
1,0.070,0.0
2,0.140,1.0
3,0.300,5.0
4,0.700,7.0
5,1.000,8.0
6,2.000,11.0
7,3.000,15.0
8,4.000,15.0
9,5.000,16.0


In [46]:
num_detected_events = np.array([sum(detect_topk_events(abs_s, r)) for r in ratios])
num_detected_events

array([ 0.,  0.,  2.,  2.,  2.,  3.,  7., 10., 12., 14.])

In [ ]:
num_detected_events = np.array([sum(detect_topk_events(abs_s, r)) for r in ratios])